In [75]:
import tensorflow.keras as keras

from neural_collaborative_filtering.Dataset import Dataset
from neural_collaborative_filtering.evaluate import evaluate_model

In [2]:
keras.__version__

'2.2.4-tf'

In [ ]:
# 논문저자 코드 참고

In [3]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    num_items = train.shape[1]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [4]:
dataset = Dataset("./neural_collaborative_filtering/Data/ml-1m")
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

In [ ]:
# pretrain된 gmf, mlp 불러오고..합쳐서 neumf의 input으로 사용.

In [58]:
epochs = 20
batch_size = 256
learning_rate = 0.001
num_negatives = 4
topK = 10
evaluation_threads = 1

# for gmf
latent_dim = 8

# for mlp
embedding_size = 64

# regs = [0, 0]

# 모델 확인용.. data를 실제로 넣어서 학습할때는 없애야함.
# user_num = 100
# item_num = 100
user_num = train.shape[0]
item_num = train.shape[1]

In [6]:
# gmf
# user, item을 동일한 size의 latent space로.. embedding..후 element wise product..
user_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_gmf")
item_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_gmf")


user_embedding_gmf = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = latent_dim,
                                            name = "user_embedding_gmf",
#                                             embeddings_initializer = "uniform",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_gmf = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = latent_dim,
                                            name = "item_embedding_gmf",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_gmf = keras.layers.Flatten()(user_embedding_gmf(user_input_gmf))
item_latent_gmf = keras.layers.Flatten()(item_embedding_gmf(item_input_gmf))

element_wise_product = keras.layers.multiply([user_latent_gmf, item_latent_gmf])

prediction_gmf = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction_gmf")(element_wise_product)

model_gmf = keras.models.Model(inputs = [user_input_gmf, item_input_gmf],
                               outputs = prediction_gmf)

In [8]:
model_gmf.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input_gmf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input_gmf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding_gmf (Embedding)  (None, 1, 8)         48320       user_input_gmf[0][0]             
__________________________________________________________________________________________________
item_embedding_gmf (Embedding)  (None, 1, 8)         29648       item_input_gmf[0][0]             
______________________________________________________________________________________________

In [7]:
# mlp
# user, item을 임베딩한후..concat, mlp layer 통과후 prediction 추출

user_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_mlp")
item_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_mlp")

user_embedding_mlp = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "user_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_mlp = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "item_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_mlp = keras.layers.Flatten()(user_embedding_mlp(user_input_mlp)) # 32
item_latent_mlp = keras.layers.Flatten()(item_embedding_mlp(item_input_mlp)) # 32

concat = keras.layers.concatenate([user_latent_mlp, item_latent_mlp]) # 64

mlp_1 = keras.layers.Dense(units = embedding_size / 2, activation = "relu", name = "mlp_1")(concat)
mlp_2 = keras.layers.Dense(units = embedding_size / 4, activation = "relu", name = "mlp_2")(mlp_1)
mlp_3 = keras.layers.Dense(units = embedding_size / 8, activation = "relu", name = "mlp_3")(mlp_2)

prediction_mlp = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction_mlp")(mlp_3)

model_mlp = keras.models.Model(inputs = [user_input_mlp, item_input_mlp],
                               outputs = prediction_mlp)

In [9]:
model_mlp.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input_mlp (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input_mlp (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding_mlp (Embedding)  (None, 1, 32)        193280      user_input_mlp[0][0]             
__________________________________________________________________________________________________
item_embedding_mlp (Embedding)  (None, 1, 32)        118592      item_input_mlp[0][0]             
____________________________________________________________________________________________

In [ ]:
# NeuMF

In [13]:
user_input_neumf = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_neumf")
item_input_neumf = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_neumf")

# gmf part
user_embedding_gmf_neumf = keras.layers.Embedding(input_dim = user_num,
                                                  output_dim = latent_dim,
                                                  name = "user_embedding_gmf_neumf",
                                                  embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                           stddev = 0.01),
#                                                   embeddings_regularizer = 0,
                                                  input_length = 1)

item_embedding_gmf_neumf = keras.layers.Embedding(input_dim = item_num,
                                                  output_dim = latent_dim,
                                                  name = "item_embedding_gmf_neumf",
                                                  embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                           stddev = 0.01),
#                                                   embeddings_regularizer = 0,
                                                  input_length = 1)

user_latent_gmf_neumf = keras.layers.Flatten()(user_embedding_gmf_neumf(user_input_neumf))
item_latent_gmf_neumf = keras.layers.Flatten()(item_embedding_gmf_neumf(item_input_neumf))

element_wise_product_neumf = keras.layers.multiply([user_latent_gmf_neumf, item_latent_gmf_neumf])

########################################################################################
# mlp part
user_embedding_mlp_neumf = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "user_embedding_mlp_neumf",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_mlp_neumf = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "item_embedding_mlp_neumf",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_mlp_neumf = keras.layers.Flatten()(user_embedding_mlp_neumf(user_input_neumf)) # 32
item_latent_mlp_neumf = keras.layers.Flatten()(item_embedding_mlp_neumf(item_input_neumf)) # 32

concat_neumf = keras.layers.concatenate([user_latent_mlp_neumf, item_latent_mlp_neumf]) # 64

mlp_1_neumf = keras.layers.Dense(units = embedding_size / 2, activation = "relu", name = "mlp_1_neumf")(concat_neumf)
mlp_2_neumf = keras.layers.Dense(units = embedding_size / 4, activation = "relu", name = "mlp_2_neumf")(mlp_1_neumf)
mlp_3_neumf = keras.layers.Dense(units = embedding_size / 8, activation = "relu", name = "mlp_3_neumf")(mlp_2_neumf)





input_neumf = keras.layers.concatenate([element_wise_product_neumf, mlp_3_neumf])

prediction_neumf = keras.layers.Dense(1,
                                      activation = "sigmoid",
                                      kernel_initializer = "lecun_uniform",
                                      name = "prediction_neumf")(input_neumf)

model_neumf = keras.models.Model(inputs = [user_input_neumf, item_input_neumf],
                                 outputs = prediction_neumf)

In [14]:
model_neumf.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input_neumf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input_neumf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding_mlp_neumf (Embed (None, 1, 32)        193280      user_input_neumf[0][0]           
__________________________________________________________________________________________________
item_embedding_mlp_neumf (Embed (None, 1, 32)        118592      item_input_neumf[0][0]           
____________________________________________________________________________________________

In [ ]:
# 이렇게 만드는거 매우 귀찮음.. class로 코드짜는게 괜히 짜는게 아니다..

In [ ]:
# load pretrained weights
model_gmf.load_weights("./model/model_gmf.h5")
model_mlp.load_weights("./model/model_mlp.h5")

In [16]:
weight_1 = model_gmf.get_layer(name = "user_embedding_gmf").get_weights()
weight_2 = model_gmf.get_layer(name = "item_embedding_gmf").get_weights()
weight_3 = model_gmf.get_layer(name = "prediction_gmf").get_weights()

weight_4 = model_mlp.get_layer(name = "user_embedding_mlp").get_weights()
weight_5 = model_mlp.get_layer(name = "item_embedding_mlp").get_weights()
weight_6 = model_mlp.get_layer(name = "mlp_1").get_weights()
weight_7 = model_mlp.get_layer(name = "mlp_2").get_weights()
weight_8 = model_mlp.get_layer(name = "mlp_3").get_weights()
weight_9 = model_mlp.get_layer(name = "prediction_mlp").get_weights()

In [50]:
pred_weight_w = np.concatenate((weight_3[0], weight_9[0]), axis = 0)
pred_weight_b = (weight_3[1] + weight_9[1]) / 2
pred_weight = [pred_weight_w, pred_weight_b]

In [51]:
model_neumf.get_layer(name = "user_embedding_gmf_neumf").set_weights(weight_1)
model_neumf.get_layer(name = "item_embedding_gmf_neumf").set_weights(weight_2)

model_neumf.get_layer(name = "user_embedding_mlp_neumf").set_weights(weight_4)
model_neumf.get_layer(name = "item_embedding_mlp_neumf").set_weights(weight_5)
model_neumf.get_layer(name = "mlp_1_neumf").set_weights(weight_6)
model_neumf.get_layer(name = "mlp_2_neumf").set_weights(weight_7)
model_neumf.get_layer(name = "mlp_3_neumf").set_weights(weight_8)

# prediction layer는 gmf, mlp trade off 존재하는데.. 0.5씩 곱해서 weight설정함..
model_neumf.get_layer(name = "prediction_neumf").set_weights(pred_weight)

In [52]:
model_neumf.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
                    loss = "binary_crossentropy")

In [54]:
user_input, item_input, labels = get_train_instances(train, num_negatives)

In [55]:
hist = model_neumf.fit([np.array(user_input), np.array(item_input)],
                       np.array(labels),
                       batch_size = batch_size,
                       epochs = epochs,
                       verbose = 1,
                       shuffle = True)

Train on 4970845 samples
Epoch 1/20
4970845/4970845 [==============================] - 58s 12us/sample - loss: 0.3177
Epoch 2/20
4970845/4970845 [==============================] - 57s 12us/sample - loss: 0.2732
Epoch 3/20
4970845/4970845 [==============================] - 60s 12us/sample - loss: 0.2617
Epoch 4/20
4970845/4970845 [==============================] - 59s 12us/sample - loss: 0.2553
Epoch 5/20
4970845/4970845 [==============================] - 60s 12us/sample - loss: 0.2505
Epoch 6/20
4970845/4970845 [==============================] - 60s 12us/sample - loss: 0.2464
Epoch 7/20
4970845/4970845 [==============================] - 60s 12us/sample - loss: 0.2429
Epoch 8/20
4970845/4970845 [==============================] - 60s 12us/sample - loss: 0.2398
Epoch 9/20
4970845/4970845 [==============================] - 59s 12us/sample - loss: 0.2369
Epoch 10/20
4970845/4970845 [==============================] - 59s 12us/sample - loss: 0.2343
Epoch 11/20
4970845/4970845 [===============

In [77]:
(hits, ndcgs) = evaluate_model(model_neumf, testRatings, testNegatives, topK, evaluation_threads)
hits, ndcgs

([1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,


In [79]:
np.array(hits).mean(), np.array(ndcgs).mean()

(0.633774834437086, 0.37051792461740063)

In [ ]:
# class로 구현하는건 구현자체는 조금 귀찮을지몰라도 나중에 써먹기엔 진짜좋다..
# model.sequental() , add() 이런걸 왜 사용하지않았을까..?